In [3]:
import os
import shutil
import cv2
import math
import random
import numpy as np
import datetime as dt
import tensorflow
import keras
from collections import deque
import matplotlib.pyplot as plt
import tensorflow as tf
%matplotlib inline

from sklearn.model_selection import train_test_split

from keras.layers import *
from keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model

In [5]:
# Load the saved model
from tensorflow import keras
MoBiLSTM_model = keras.models.load_model(r'C:\Users\HP\projects\ML\violence\model_MobileNet-LSTM_01.h5')

# Specify the height and width to which each video frame will be resized in our dataset.
IMAGE_HEIGHT , IMAGE_WIDTH = 64, 64

# Specify the number of frames of a video that will be fed to the model as one sequence.
SEQUENCE_LENGTH = 24

#DATASET_DIR = "/content/drive/MyDrive/violence/Real Life Violence Dataset"
CLASSES_LIST = ["NonViolence","Violence"]

In [ ]:
cap = cv2.VideoCapture(0)
frames_queue = deque(maxlen = SEQUENCE_LENGTH)
SEQUENCE_LENGTH = 25
while True:
    ret , frame = cap.read()
    if not ret:  # Check if frame was read successfully
        print("Error reading frame. Exiting loop.")
        break  
    resized_frame = cv2.resize(frame, (IMAGE_HEIGHT, IMAGE_WIDTH))
        # Normalize the resized frame.
    normalized_frame = resized_frame / 255
        # Passing the  pre-processed frames to the model and get the predicted probabilities.
    frames_queue.append(normalized_frame)
        # We Need at Least number of SEQUENCE_LENGTH Frames to perform a prediction.
        # Check if the number of frames in the queue are equal to the fixed sequence length.
    if len(frames_queue) == SEQUENCE_LENGTH:
        predicted_labels_probabilities = MoBiLSTM_model.predict(np.expand_dims(frames_queue, axis = 0))[0]
        predicted_label = np.argmax(predicted_labels_probabilities)
        predicted_class_name = CLASSES_LIST[predicted_label]
        if predicted_class_name == "Violence":
            cv2.putText(frame, predicted_class_name, (5, 100), cv2.FONT_HERSHEY_SIMPLEX, 3, (0, 0, 255), 12)
        else:
            cv2.putText(frame, predicted_class_name, (5, 100), cv2.FONT_HERSHEY_SIMPLEX, 3, (0, 255, 0), 12)
    cv2.imshow('Kya aap hinsak ho? ', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()